In [1]:
!pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers pypdf pipeline torch

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.8.2 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
cudf

In [2]:
base_model = "/kaggle/input/mistral/pytorch/7b-v0.1-hf/1"

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

2024-04-17 15:27:51.337252: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-17 15:27:51.337415: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-17 15:27:51.555713: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,  # Pass torch.bfloat16 here
        device_map="auto",
        trust_remote_code=True,
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(True, True)

In [5]:
from transformers import pipeline
pipeline = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,  # needed by langchain
    # model params
    max_new_tokens=256,
    #temperature=0.1,  # creativity of responses: 0.0 none ->  1.0 max
    repetition_penalty=1.1  # to avoid repeating output
)

In [6]:
print(type(model))


<class 'transformers.models.mistral.modeling_mistral.MistralForCausalLM'>


In [7]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(base_model); config

MistralConfig {
  "_name_or_path": "/kaggle/input/mistral/pytorch/7b-v0.1-hf/1",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.39.3",
  "use_cache": true,
  "vocab_size": 32000
}

In [8]:
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipeline)

In [9]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader('/kaggle/input/filess/Question_Answering_Dataset.pdf')
documents = loader.load_and_split()

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
texts = text_splitter.split_documents(documents)

In [11]:
# loading the FAISS vector database
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# This is a sentence-transformers model: It maps sentences & paragraphs to a 768 dimensional dense vector space
# and can be used for tasks like clustering or semantic search.
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
# storing embeddings in the vector store
vectorstore = FAISS.from_documents(texts, embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
from langchain.chains import ConversationalRetrievalChain
from langchain import PromptTemplate
tt = PromptTemplate.from_template("""Act as professor and answer very clearly to question asked, and be very clear and detailed<>
{content}]""")

chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=False)

In [13]:
query = "What is the assessment strategy for the course Information Systems and Technologies?"
chat_history = []
result = chain({"question": tt.format(content=query), "chat_history": chat_history})
def process_answer(answer):
    lines = answer.split("\n")
    for line in lines:
        if "Question:" in line:
            break
        print(line.strip())
process_answer(result['answer'])

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




The assessment strategy includes a 2 -hour written examination, a 30 -minute group presentation on emerging technologies, and a mini-project completed b y groups proposing solutions for real -case problems.



In [19]:
query = "Combien de crédits sont associés au cours Systèmes et technologies de l'information?" 
chat_history = []
result = chain({"question": tt.format(content=query), "chat_history": chat_history})
def process_answer(answer):
    lines = answer.split("\n")
    for line in lines:
        if "Question:" in line:
            break
        print(line.strip())
process_answer(result['answer'])


The course offers 4 ECTS credits.



In [20]:
query = "Qui est le vainqueur de la coupe du monde de football en 2018?"
chat_history = []
result = chain({"question": tt.format(content=query), "chat_history": chat_history})
print(result['answer'])



Question: Act as professor and answer very clearly to question asked, and be very clear and detailed<>
Qui est le vainqueur de la coupe du monde de football en 2018?]
Helpful Answer:

Question: Act as professor and answer very clearly to question asked, and be very clear and detailed<>
Qui est le vainqueur de la coupe du monde de football en 2018?]
Helpful Answer:

Question: Act as professor and answer very clearly to question asked, and be very clear and detailed<>
Qui est le vainqueur de la coupe du monde de football en 2018?]
Helpful Answer:

Question: Act as professor and answer very clearly to question asked, and be very clear and detailed<>
Qui est le vainqueur de la coupe du monde de football en 2018?]
Helpful Answer:

Question: Act as professor and answer very clearly to question asked, and be very clear and detailed<>
Qui est le vainqueur de la coupe du monde de football en 2018?]
Helpful Answer:

